In [ ]:
from model import layers, activations, networks, utils, optimizers, losses, metrics, initializations
import pandas as pd

### Prepare data
You will need to download the dataset from https://www.kaggle.com/datasets/saurabh00007/iriscsv and place it into the *data* directory.

In [ ]:
data_orig = pd.read_csv('data/iris.csv')
data = data_orig.copy()
data.drop(columns=['Id'], inplace=True)
data.head()

In [ ]:
data.describe()

In [ ]:
data.info()

The labels are categorical values. To be used in the model, all data needs to be numerical. The function `categorical_to_numeric()` can be used to one-hot-encode all categorical data of a Pandas DataFrame object.

In [ ]:
data_enc = utils.categorical_to_numeric(data)
data_enc.sample(10)

Next the data is split into a training and a testing dataset using the `split_train_test_data()` to evaluate the model later on. Before splitting the data is also shuffled, since sometimes raw data is sorted in some way.

In [ ]:
array = data_enc.to_numpy()
train_array, test_array = utils.split_train_test_data(array, ratio=0.2)
train_array[:5]

Features and labels are now seperated.

In [ ]:
x_train, y_train = utils.split_X_Y(train_array, 4)
x_test, y_test = utils.split_X_Y(test_array, 4)

Neural networks tend to run into problems if values are very high. Therefore it is common to normalize the data. This can be done using the `normalize()` function, which applies min-max feature scaling to an array.<br><br>
$ X'=a+\frac{(X-X_{min})\cdot(b-a)}{X_{max}-X_{min}} $<br><br>, where<br><br>$ a $ ... lower bound<br>$ b $ ... upper bound

In [ ]:
x_train = utils.normalize(x_train)
x_test = utils.normalize(x_test)
x_train[:5]

Lastly the arrays are reshaped to fit the models' dimension requirement.

In [ ]:
x_train = utils.expand_dims(x_train, 4)
y_train = utils.expand_dims(y_train, 2)
x_test = utils.expand_dims(x_test, 4)
y_test = utils.expand_dims(y_test, 2)

print (f'{x_train.shape=}')
print (f'{y_train.shape=}')
print (f'{x_test.shape=}')
print (f'{y_test.shape=}')

### Build the neural network structure
Here the individual layers of the neural network models are defined. Every model also needs an *Input* and *Output* layer. Other layers and their parameters can be chosen arbitrarily.

In [ ]:
model = networks.FeedForward(input_shape=(4,), layers=[
    layers.Dense(nr_neurons=32, activation=activations.Tanh),
    layers.Dense(nr_neurons=32, activation=activations.Tanh),
    layers.Dense(nr_neurons=32, activation=activations.Tanh),
    layers.Dense(nr_neurons=3, activation=activations.Softmax)
])

The network is compiled to internally connect it's layers and initialize the model.

In [ ]:
model.compile(
    optimizer=optimizers.sgd(learning_rate=0.001),
    loss=losses.categorical_crossentropy,
    metric=metrics.Accuracy
)

In [ ]:
model.summary()

### Train the model

In [ ]:
model.train(x_train, y_train, epochs=200, log=True)

In [ ]:
model.plot_training_loss()

In [ ]:
model.plot_neuron_activations()

In [ ]:
model.plot_neuron_gradients()

### Evaluate the model

In [ ]:
model.evaluate(x_test, y_test)